In [ ]:
import sys
!{sys.executable} -m pip install seaborn

In [ ]:
import logging

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import azureml.core
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication(tenant_id = '7d9fdaba-a83f-43d1-8d9e-877fd1d8b9df')
ws = Workspace.from_config(auth = auth)

# choose a name for experiment
experiment_name = 'ml-dog_breeds-experiment'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

# Create or Attach existing AmlCompute

A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.

Creation of AmlCompute takes approximately 5 minutes.
If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "tf-compute-instance"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Data

In [ ]:
df_labels = pd.read_csv("input/labels.csv")
df_labels.head()

Add filepath to the DataFrame

In [ ]:
df_labels['filename'] = df_labels.id.map(lambda id: f'input/data/train/{id}.jpg')
df_labels.head()

# Data exploration
Show what we are ourself getting self into ...

In [ ]:
print(df_labels.describe())
print('')   
print('----------------------------------------------------')
print('')
print(df_labels.breed.describe())

In [ ]:
unique_labels = df_labels.breed.unique()
print('Unique amount of labels', len(unique_labels))

We have good news - all breeds are present in train dataset.

So lets calculate amount if images for every breed and check most frequent.

In [ ]:
gr_labels = df_labels.groupby("breed").count()
gr_labels = gr_labels.rename(columns = {"id" : "count"})
gr_labels = gr_labels.sort_values("count", ascending=False)
gr_labels.head(10)
# gr_labels.tail()

In [ ]:
# plt.figure(figsize=(20,20))
ax = sns.barplot(x=df_labels.breed.unique()[:10], y=df_labels.breed.value_counts()[:10])
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
ax = sns.scatterplot(x=df_labels.breed.unique(), y=df_labels.breed.value_counts())
ax.set_xticklabels('')
plt.tight_layout()
plt.show()

# Data processing

In [ ]:
[df_labels.filename.values.tolist()[:10]]

Converting every image to a array of Tensors, with image size 256x256 pixels

In [ ]:
from tqdm import tqdm

img_pixel = np.array([img_to_array(load_img(img, target_size=(224, 224))) for img in tqdm(df_labels.filename.values.tolist() ) ])

Converting image labels to auto-hot encoding

In [ ]:
img_label = df_labels.breed
img_label = pd.get_dummies(df_labels.breed)
img_label.head()

In [ ]:
# img_label.to_csv('breed-columns.csv')

In [ ]:
print('original label:', df_labels.breed.values[0])
print('index where label occurs in hot-encoding array:', img_label.values[0].argmax())
print()

print('there will be a 1 where the sample label occurs:')
print(img_label.values[0].astype(int))
# print(np.where(df_labels.breed.unique == img_label))

Creating the train numpy arrays (X)=images and (Y)=breed of image, therefore dog

In [ ]:
X = img_pixel
y = img_label.values
print(X.shape)
print(y.shape)

In [ ]:
# Split arrays or matrices into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train = X_train.repeat()
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Takes data & label arrays, generates batches of augmented data.
train_generator = train_datagen.flow(x=X_train, y=y_train, batch_size=32, seed=42)

In [ ]:
plt.figure(figsize=(8,8))
for image in range(0, 3):
    plt.subplot(1, 3, image+1)
    for X_batch, Y_batch in train_generator:
        image = X_batch[0]
        plt.title(img_label.columns[Y_batch[0].argmax()])
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

# Takes data & label arrays, generates batches of augmented data.
validation_generator = validation_datagen.flow(x=X_test, y=y_test, batch_size=32, seed=42)

In [ ]:
plt.figure(figsize=(8,8))
for image in range(0, 3):
    plt.subplot(1, 3, image+1)
    for X_batch, Y_batch in validation_generator:
        image = X_batch[0]
        plt.title(img_label.columns[Y_batch[0].argmax()])
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

# Train on Microsoft Azure Machine Learning

With this section of the notebook, we are going to define what and how we are going to train a model on a Azure Machine Learning cloud service. We are going to do this by leveraging the Azure SDK and therefore their capabilities.

In [ ]:
from azureml.core import Dataset, Datastore


In [ ]:
training_data, validation_data = dogs_ds.random_split(percentage=0.8, seed=42)
label_column_name = 'breed'

In [ ]:
automl_settings = {
    "primary_metric": 'average_precision_score_weighted',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 2, # This is a limit for testing purpose, please increase it as per cluster size
    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    task = 'classification',
    debug_log = 'automl_errors.log',
    compute_target = compute_target,
    training_data = training_data,
    label_column_name = label_column_name
)

# automl_config = AutoMLConfig(task='classification', debug_log='automl_errors.log', compute_target=compute_target, X=X_train, y=y_train, X_valid=X_test, y_valid=y_test)

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)

In [ ]:
remote_run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

In [ ]:
# Analyze results via Azure Portal

In [ ]:
# WRITE HERE 

# Train on TensorFlow (locally)

For the record. You can also run this in the cloud. But in case you are running this in Azure, I would recommend to run the previous part, there we are levering the Azure Cloud SDK.

# Building the model

Before we can build the model, there are a few things we need to define:

- The input shape (images, in the form of Tensors) to our model
- The output shape (image labels, in the form of Tensors) of our model
- The URL of the model we want to use (because of Transfer Learning)

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, 224, 224, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(df_labels.breed.unique()) # number of unique labels

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, ZeroPadding2D, Flatten, Dropout, MaxPooling2D

In [ ]:
def create_model_transfer_learning(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
    print("Building model with:", MODEL_URL)

    # setup the model layers
    model = Sequential([hub.KerasLayer(MODEL_URL), Dense(units=OUTPUT_SHAPE, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.build(INPUT_SHAPE)

    return model

def create_model_scratch(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE):
    print("Building own made model")

    model = Sequential()
    model.add(Conv2D(32, (1, 1), activation='relu', input_shape=(224, 224, 3)))

    model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(OUTPUT_SHAPE, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model_transfer_learning()
model.summary()

In [ ]:
own_model = create_model_scratch()
own_model.summary()

# Creating callbacks (things to help our model)
We've got a model ready to go but before we train it we'll make some callbacks

Callbacks are helper functions a model can use during training to do things such as save a models progres, check a models progress or stop training early if a model stops improving.

## Tensorboard Callback

TensorBoard helps provide a visual way to monitor the progress of your model during and after training.

It can be used directly in a notebook to track the performance measures of a model such as loss and accuracy.

To set up a TensorBoard callback and view TensorBoard in a notbook, we need to do three things:

1. Load the TensorBoard notebook extension.

2. Create a TensorBoard callback which is able to save logs to a directory and pass it to our model's fit() function.

3. Visualize the our models trainigs logs using %tensorboard magic function (we'll do this later on)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime
import os

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
    # Create a log directly for storing TensorBoard logs
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%d%m%Y-%H%M%S'))

    return tf.keras.callbacks.TensorBoard(logdir)

## Early Stopping Callback

Early stopping helps prevent overfitting by stopping a model when a certain evaluation metric stops improving. If a model trains for too long, it can do so well at finding patterns in a certain dataset that it's not able to use those patterns on another dataset it hasn't seen before (doesn't generalize).

It's basically like saying to our model, "keep finding patterns until the quality of those patterns starts to go down."

Therefore, when there is not enough change in our model, we can say that the model have reached a equilibrium.

In [ ]:
# Create early stopping (once our model stops improving, stop training)
# it stops after 3 rounds of no improvements
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

# Uncomment if learning rate reductions to model needs to be added.
# Also add it the the model.fit_generator function.
# learning_rate_reduction = ReduceLROnPlateau(
#     monitor='val_accuracy', 
#     patience=2, 
#     verbose=1, 
#     factor=0.5, 
#     min_lr=0.0001
# )

# callbacks = [early_stopping, learning_rate_reduction]

## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows you to continually save the model both during and at the end of training.

Saves every 5 epoch a checkpoint

In [ ]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1, save_freq=5)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

## Load weigts from checkpoint

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# Loads the weights
model.load_weights(latest)

## Training a model

The final parameter we'll define before training is NUM_EPOCHS (also known as number of epochs).

NUM_EPOCHS defines how many passes of the data we'd like our model to do. A pass is equivalent to our model trying to find patterns in each dog image and see which patterns relate to each label.

If NUM_EPOCHS=1, the model will only look at the data once and will probably score badly because it hasn't a chance to correct itself. It would be like you competing in the international hill descent championships and your friend Adam only being able to give you 1 single instruction to get down the hill.

What's a good value for NUM_EPOCHS?

This one is hard to say. 10 could be a good start but so could 100. This is one of the reasons we created an early stopping callback. Having early stopping setup means if we set NUM_EPOCHS to 100 but our model stops improving after 22 epochs, it'll stop training.

In [ ]:
# How many rounds should we get the model to look through the data?
NUM_EPOCHS = 15

Create a model using create_model().

Setup a TensorBoard callback using create_tensorboard_callback() (we do this here so it creates a log directory of the current date and time).

Call the fit() function on our model passing it the training data, validatation data, number of epochs to train for and the callbacks we'd like to use.

Return the fitted model.

In [ ]:
def train_model():
    # Trains a given model and returns the trained version

    model = create_model_transfer_learning()

    tensorboard = create_tensorboard_callback() # Change this function if you want to use own-model (not transfer learning)

    model.fit_generator(
        generator=train_generator, 
        # steps_per_epoch= X_train.shape[0] // 32, 
        validation_data=validation_generator, 
        validation_freq=1,  
        # validation_steps= y_train.shape[0] // 32,
        epochs=NUM_EPOCHS, 
        verbose=1,
        callbacks=[tensorboard, early_stopping, cp_callback])

    return model

In [ ]:
model = train_model()

# Save model or Load model

Call model.save to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

In [ ]:
# Save complete model as new format.
# model.save('saved_model/my_model')

# Save model as old Keras format. H5 format.
# model.save('saved_model/h5_model.h5')

# Save model as json format. (Use this for Streamlit)
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("saved_model/model.h5")

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('saved_model/my_model')

# Checking the TensorBoard logs

Now our model has been trained, we can make its performance visual by checking the TensorBoard logs.

The TensorBoard magic function (%tensorboard) will access the logs directory we created earlier and viualize its contents.


Thanks to our early_stopping callback, the model stopped training after 26 or so epochs (in my case, yours might be slightly different). This is because the validation accuracy failed to improve for 3 epochs.

But the good new is, we can definitely see our model is learning something. The validation accuracy got to 65% in only a few minutes.

This means, if we were to scale up the number of images, hopefully we'd see the accuracy increase.

To see the logs visit : http://localhost:6006 in your browser

In [ ]:
%tensorboard --logdir logs

# Making and evaluating predictions using a trained model

Before we scale up and train on more data, let's see some other ways we can evaluate our model. Because although accuracy is a pretty good indicator of how our model is doing, it would be even better if we could could see it in action.

Making predictions with a trained model is as calling predict() on it and passing it data in the same format the model was trained on.

In [ ]:
# Make predictions on the validation data (not used to train on)
predictions = model.predict(validation_generator, verbose=1) # verbose shows us how long there is to go
predictions

In [ ]:
# Check the shape of predictions
predictions.shape

Making predictions with our model returns an array with a different value for each label.

In this case, making predictions on the validation data (2045 images) returns an array (predictions) of arrays, each containing 120 different values (one for each unique dog breed).

These different values are the probabilities or the likelihood the model has predicted a certain image being a certain breed of dog. The higher the value, the more likely the model thinks a given image is a specific breed of dog.

Let's see how we'd convert an array of probabilities into an actual label.


In [ ]:
# First prediction
# print(predictions[0])
print(f'Max value (probability of prediction): {np.max(predictions[0])}')
print(f'Sum: {np.sum(predictions[0])}')                                     # because we used softmax activation in our model, this will be close to 1
print(f'Max index: {np.argmax(predictions[0])}')                            # the index of where the max value in predictions[0] occurs
print(f'Predicted label: {img_label.columns[np.argmax(predictions[0])]}')   # the predicted label

Having this information is great but it would be even better if we could compare a prediction to its true label and original image.

To help us, let's first build a little function to convert prediction probabilities into predicted labels.

Note: Prediction probabilities are also known as confidence levels.


In [ ]:
# Turn predictions probabilities into their respective label (this is then easier to understand)
def get_pred_label(prediction_probabilities):
    """
    Turns an array of prediction probabilities into a label.
    """

    return img_label.columns[np.argmax(prediction_probabilities)]

In [ ]:
# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[0])
pred_label

Now we've got a list of all different predictions our model has made, we'll do the same for the validation images and validation labels.

Remember, the model hasn't trained on the validation data, during the fit() function, it only used the validation data to evaluate itself. So we can use the validation images to visually compare our models predictions with the validation labels.

In [ ]:
dataset = tf.data.Dataset.from_generator(lambda: validation_generator, (tf.float32, tf.float32))
dataset = dataset.unbatch()

images = []
labels = []
d = list(dataset.take(1000).as_numpy_iterator())
for image, label in d:
    images.append(image)
    labels.append(get_pred_label(label))

# for i, l in list(dataset.as_numpy_iterator()):
    # print(i, l)

We got ways to get:

- Prediction labels
- Validation labels (truth labels)
- Validation images

More specifically, we want to be able to view an image, its predicted label and its actual label (true label).

The first function we'll create will:
- Take an array of prediction probabilities, an array of truth labels, an array of images and an integer.
- Convert the prediction probabilities to a predicted label.
- Plot the predicted label, its predicted probability, the truth label and target image on a single plot.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
    # show the prediction, ground truth label and image for sample n

    pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
    # print(pred_prob, true_label, image)

    # Get the predicted label
    pred_label = get_pred_label(pred_prob)

    # Plot image & remove ticks
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])

    # Change the color of the title depending on if the prediction is right or wrong
    if pred_label == true_label:
        color = "green"
    else:
        color = "red"

    plt.title("{} {:2.0f}% ({})".format(pred_label, np.max(pred_prob)*100, true_label), color=color)
    

In [ ]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions, labels=labels, images=images)

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plots the top 10 highest prediction confidences along with
  the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
  # Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]
  # Find the top 10 prediction labels
  top_10_pred_labels =  unique_labels[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)), top_10_pred_values, color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)), labels=top_10_pred_labels, rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions, labels=labels, n=9)

In [ ]:
# Let's check a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols

plt.figure(figsize=(5*2*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions, labels=labels, images=images, n=i+i_multiplier)
  
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions, labels=labels, n=i+i_multiplier)

plt.tight_layout(h_pad=1.0)
plt.show()


# Making predictions on the test dataset

To make predictions on the test data, we'll:

    Get the test image filenames.
    Make a predictions array by passing the test data to the predict() function

In [ ]:
test_path = 'input/data/test/'
test_filenames = [test_path + fname for fname in os.listdir(test_path)]

test_filenames[:3]

## How many test images are there?

In [ ]:
len(test_filenames)

In [ ]:
IMG_SIZE = 224

def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [ ]:
# Create test data
print("Creating test data batches...")
data = tf.data.Dataset.from_tensor_slices((tf.constant(test_filenames))) # only filepaths
data_batch = data.map(process_image).batch(32)
data_batch

Note: Since there are 10,000+ test images, making predictions could take a while, even on a GPU. So beware running the cell below may take up to an hour.

In [ ]:
test_data_batch = data_batch
# Make predictions on test data batch using the loaded full model
test_predictions = model.predict(test_data_batch, verbose=1)

In [ ]:
# Show the first 10 predictions
test_predictions[:10]

In [ ]:
get_pred_label(test_predictions[0])
test_filenames[0]

In [ ]:
plt.imshow(load_img(test_filenames[0]))
plt.title(get_pred_label(test_predictions[0]))

# Preparing test dataset predictions for CVS export

To get the data in this format, we'll:

- Create a pandas DataFrame with an ID column as well as a column for each dog breed.
- Add data to the ID column by extracting the test image ID's from their filepaths.
- Add data (the prediction probabilities) to each of the dog breed columns using the unique_breeds list and the test_predictions list.
- Export the DataFrame as a CSV to submit it.


In [ ]:
# Create pandas DataFrame with empty columns
preds_df = pd.DataFrame(columns=["id"] + list(unique_labels))
preds_df.head()

In [ ]:
# Append test image ID's to predictions DataFrame
preds_df["id"] = [os.path.splitext(path)[0] for path in os.listdir(test_path)]
preds_df.head()

In [ ]:
# Add the prediction probabilities to each dog breed column
preds_df[list(unique_labels)] = test_predictions
preds_df.head()

In [ ]:
preds_df.to_csv("submission_with_mobilienetV2.csv", index=False)